In [1]:
import pandas as pd
import numpy as np
import os, sys

os.getcwd()
rpath = os.path.abspath('../')
if rpath not in sys.path:
    sys.path.insert(0,rpath)

In [2]:
path = os.path.join(rpath, 'data', 'clean_data.csv')
df = pd.read_csv(path)
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14841,NaN,2024-09-26 12:24:51+00:00,photos\@sinayelj_14841.jpg
1,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14840,NaN,2024-09-26 12:24:51+00:00,photos\@sinayelj_14840.jpg
2,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14839,ለኮንዶሚኒየም ለጠባብ ቤቶች ገላግሌ የሆነ ከንፁህ የሲልከን ጥሬ እቃ የተ...,2024-09-26 12:24:51+00:00,photos\@sinayelj_14839.jpg
3,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14838,NaN,2024-09-26 12:24:12+00:00,photos\@sinayelj_14838.jpg
4,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,14837,NaN,2024-09-26 12:24:12+00:00,photos\@sinayelj_14837.jpg
...,...,...,...,...,...,...
4618,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,599,NaN,2021-04-16 18:12:10+00:00,photos\@sinayelj_599.jpg
4619,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,598,Baby potty\n0905707448\n0945097042,2021-04-16 18:12:10+00:00,photos\@sinayelj_598.jpg
4620,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,197,NaN,2020-11-26 18:30:52+00:00,NaN
4621,SINA KIDS/ሲና ኪድስⓇ,@sinayelj,182,ውድ የሲና ኪድስ ደምበኞች በድጋሚ ገብቷል \nየመዋኛ ገንዳ ትልቅ ሳይዝ ...,2020-11-10 06:59:31+00:00,photos\@sinayelj_182.jpg


In [3]:
import nltk
nltk.download('punkt')  # Download necessary resources

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nolaw\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.char_classes import ALPHA
from spacy.util import compile_infix_regex
import pandas as pd

# Load a base model (e.g., English) to use for custom tokenization
nlp = spacy.blank('en')

# Customize the tokenizer
infixes = (ALPHA,)  # Use alpha characters only (basic Amharic chars)
infix_re = compile_infix_regex(infixes)
nlp.tokenizer = Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)


# Tokenize each message using spaCy
df['tokenized'] = df['Message'].apply(lambda x: [token.text for token in nlp(x)])

print(df[['Message', 'tokenized']])

ModuleNotFoundError: No module named 'spacy'

In [ ]:
df.head()

In [ ]:
import re

def label_message_utf8_with_birr(message):
    # Split the message at the first occurrence of '\n'
    if '\n' in message:
        first_line, remaining_message = message.split('\n', 1)
    else:
        first_line, remaining_message = message, ""
    
    labeled_tokens = []
    
    # Tokenize the first line
    first_line_tokens = re.findall(r'\S+', first_line)
    
    # Label the first token as B-PRODUCT and the rest as I-PRODUCT
    if first_line_tokens:
        labeled_tokens.append(f"{first_line_tokens[0]} B-PRODUCT")  # First token as B-PRODUCT
        for token in first_line_tokens[1:]:
            labeled_tokens.append(f"{token} I-PRODUCT")  # Remaining tokens as I-PRODUCT
    
    # Process the remaining message normally
    if remaining_message:
        lines = remaining_message.split('\n')
        for line in lines:
            tokens = re.findall(r'\S+', line)  # Tokenize each line while considering non-ASCII characters
            
            for token in tokens:
                # Check if token is a phone number (e.g., 10 consecutive digits)
                if re.match(r'^\d{10}$', token):
                    labeled_tokens.append(f"{token} I-PHONE")  # Label as I-PHONE
                
                # Check if token is a price (e.g., 500 ETB, $100, or ብር)
                elif re.match(r'^\d+(\.\d{1,2})?$', token) or 'ETB' in token or 'ዋጋ' in token or '$' in token or 'ብር' in token or 'birr' in token:
                    labeled_tokens.append(f"{token} I-PRICE")
                
                # Check if token could be a location (e.g., cities or general location names)
                elif any(loc in token for loc in ['Addis Ababa', 'ለቡ', 'ለቡ መዳህኒዓለም', 'መገናኛ', 'ቦሌ', 'ሜክሲኮ', 'ብስራተ', 'ገብርኤል', 'ገርጂ', 'ኢምፔሪያል', 'ህንፃ', 'ፕላዛ', '4ኪሎ', 'ፎቅ', 'ላፍቶ', 'ሞል', 'ስላሴ']):
                    labeled_tokens.append(f"{token} I-LOC")
                
                # Assume other tokens are part of a product name or general text
                else:
                    labeled_tokens.append(f"{token} O")
    
    return "\n".join(labeled_tokens)

# Apply the updated function to the non-null messages
df['Labeled_Message'] = df['Message'].apply(label_message_utf8_with_birr)

# Display the updated DataFrame
df.head()

In [ ]:
# Save the updated labeled dataset to a file in CoNLL format
labeled_data_birr_path = os.path.join(rpath, 'data', 'labeled_telegram_product_price_location.txt')
with open(labeled_data_birr_path, 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        f.write(f"{row['Labeled_Message']}\n\n")

In [ ]:
# Define the data path
data_path = os.path.join(rpath, 'data', 'labeled_telegram_product_price_location.txt')

# Try specifying the encoding
with open(data_path, 'r', encoding='utf-8', errors='replace') as file:
    lines = file.readlines()

# Process lines as needed, assuming you are using tab as the delimiter
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter

# Create a DataFrame, assuming the first column is 'Labeled_Message' and second for labels
labeled_data = pd.DataFrame(data, columns=['Labeled_Message'])

# Display the DataFrame
print(labeled_data.head())

In [ ]:
# Initialize lists for tokens and labels
tokens = []
labels = []

# Iterate over the DataFrame to split the 'Labeled_Message' column
for message in labeled_data['Labeled_Message']:
    parts = message.split()  # Split the string by whitespace
    if len(parts) == 2:      # Ensure that line has exactly two parts
        token = parts[0]      # The token (first part)
        label = parts[1]      # The label (second part)
        tokens.append(token)
        labels.append(label)

# Create a new DataFrame with the separated tokens and labels
df_labeled = pd.DataFrame({
    'Token': tokens,
    'Label': labels
})

# Display the DataFrame
print(df_labeled.head())